<center>
<h1>Detección y conteo de vulgaridades en textos en Español con spanlp</h1><br>
</center>
<img src="static/imagen.png" alt="spanlp">

**Problema:**
<br><br>
El archivo tweets.csv contiene trinos de personas que comentan 
sobre el servicio de un operador de telefonía. Se desea saber si contiene o no vulgaridades y cuantas son.
<br><br>
**Objetivos:**
<br>
* Leer el archivo de datos que contiene los tweets.
* Detectar e indicar si los datos tienen vulgaridades o no.
* Contar la cantidad de vulgaridades que hay.

In [3]:
def leer_archivo(nombre_archivo:str='', separador_datos:str='', columna:str=''):
    tweets = []
    with open(nombre_archivo, encoding='utf-8') as data:
        reader = csv.DictReader(data, delimiter=separador_datos)
        for fila in reader:
            tweets.append(fila[columna])
    return tweets

In [5]:
def validar_vulgaridades(data:list=None):
    if data is None or len(data) == 0:
        raise Exception("Oiga! para validar vulgaridades debe enviar datos. esta enviando datos nulos o vacios.")
    
    contador_palabrotas = 0
    
    for texto in data:
        if isinstance(texto, str):
            es_palabrota = validador.contains_palabrota(texto)
            if es_palabrota:
                contador_palabrotas += 1
    return (contador_palabrotas>0,contador_palabrotas)

In [10]:
comments = leer_archivo(nombre_archivo='tweets.csv', separador_datos='|', columna='comment_1')
contiene, cantidad = validar_vulgaridades(comments)

print(f"Los datos contienen {cantidad} vulgaridades." if contiene else "Los datos no contienen vulgaridades.")

Los datos contienen 7 vulgaridades.
